# Planet Destinations Python Client Introduction

This tutorial is an introduction to [Planet's](https://www.planet.com) Destination API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Destination API allows you manage and securely store cloud storage bucket credentials as a 'Destination' for a product delivery on the Planet Platform. This can service as an alternative to adding credentials to multiple workflows, for example, as the Destination can be setup once, and then referenced in different Order or Subscription requests.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Destinations Documentation](https://docs.planet.com/develop/apis/destinations/)

The basic workflow for interaction with the Destinations API is:
1. Prepare destination variables; mainly credentials and bucket name, with some other items depending on cloud storage provider.
2. Create Destination using the Destinations API
3. Use the Destination Reference ID in Orders and Subscription Requests.

____

## <u>Destination Information</u>

#### Supported Cloud Storage Destinations:
* Amazon S3
    * Any other service that implements the S3 compatibility API
* Google Cloud Storage
* Microsoft Azure Blob Storage
* Oracle Cloud Storage


#### Detinations Limitations:
* Each *organization* is provided a limitation of up to 50 Destinations, which will be both visible and usable by each individual within said organization.
* The creator of a Destination is considered its *owner*, and, along with your organizational administrator, will be the only one allowed to modify a Destination.
* All Destinations must have unique names.
* All Destinations are rate limited to 3 requests per second.

#### Destination Credentials
* Credentials used to create a Destination must have both `write` and `delete` permissions. 
    * When a destination is created or updated, Planet verifies access by attempting to write and then delete a test file named `planetverify.txt`. 
    * If the permissions are correctly configured, this file will be removed immediately and will not appear in the storage bucket or container.* If you ever see this file, your credentials may be lacking the `delete` permissions needed.

#### Credential Security
* Destination credentials are treated as secrets which are encrypted at rest and in transit between Planet systems, accessed and decrypted only when strictly required, during a delivery.
* Destinations API will always redact these secrets in its responses.
    * You can update credentials via the Destination API, but never read them using it.


____

## Setup

In order to interact with the Planet Destinations API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

### Imports

In [1]:
import json
import pathlib
import planet
import os
from datetime import datetime
from planet import Auth, Planet, Session, DestinationsClient
# Add additional imports as needed

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. If this is your first time accessing the Planet SDK, you will also be prompted first to authorize the SDK access to your Planet account. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [ ]:
# Authenticate for the Planet SDK; See docs: https://docs.planet.com/develop/authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = planet.Auth.from_profile('planet-user', save_state_to_storage=True)
auth.ensure_initialized(allow_open_browser=False, allow_tty_prompt=True)

session = planet.Session(auth)
pl = planet.Planet(session)

_____

### Prepare Destination Credentials

[Add section description here]

In [ ]:
# Add your code here

___

### Section 2

[Add section description here]

In [ ]:
# Add your code here

____

### Section 3

[Add section description here]

In [ ]:
# Add your code here

_____

### Section 4

[Add section description here]

In [ ]:
# Add your code here

___

### Section 5

[Add section description here]

In [ ]:
# Add your code here